### ZoKrates Benchmarking

#### Functionality:
This notebook benchmarks the gas costs for different .zok functions. By setting __max_inputs__ programms with __[1 - max_inputs]__ will be generated.

E.g.
__max_inputs = 3__ will generate the following programms, deploy and execute them, and plot the gas usage for each programm

```
def main(field a):
	 return a

def main(field a, field b):
	 return a, b

def main(field a, field b, field c):
	 return a, b, c
```

#### Requirements for running:
- Geth or similar blockchain client running
- ZoKrates is installed locally (via install script)
- solc is installed globally (npm i -g solc)

#### Running Geth with Istanbul HF:
- Install Geth
- Initialize private chain with custom genesis:
    ```
    geth --datadir chain init genesis.json
    ```
- Run local private chain with miners targeting 100,000,000 block gas limit: 
    ``` 
    geth --port 3000 --networkid 58342 --nodiscover --datadir="chain" --maxpeers=0 --rpc --rpcport 8545 --rpcaddr 127.0.0.1 --rpccorsdomain "*" --rpcapi "eth,net,web3" --nodiscover --allow-insecure-unlock --unlock 0xD51E15b8580D7D1dcBD8FcfD8B712E45807BC305 --password pass --etherbase 0xD51E15b8580D7D1dcBD8FcfD8B712E45807BC305 --mine --minerthreads=1   --miner.gastarget 100000000 --miner.gaslimit 100000000
    ```
    
#### Running Ganache-Cli with Muriel Glacier HF and unlimited contract size:
ganache-cli -l 100000000 -k muirGlacier --allowUnlimitedContractSize

### Installing Dependencies:


In [1]:
%%capture
import sys
!{sys.executable} -m pip install web3
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy 

### Imports

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os # paths
import shutil # file operations
from itertools import chain # operate on iterators like ranges
from web3 import Web3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.eth.defaultAccount = w3.eth.accounts[0]


### Benchmark Configuration:
Set max number of inputs for the function. proving_scheme only accepts g16 for now, since thats the only available option in the install script version of ZoKrates. Once they are implemented, they can be choosen via provided variable.

In [3]:
max_inputs = 360 # largest number of contract inputs. Max size w.r. to contract size without solc --optimize flag: 168. This is just below 2 kilobyte (2KB bounday, see EIP-170)
step = 20 # increment in number of inputs between two experiments
trend_factor = 2
proving_scheme = "g16"
target_dir = './target' # directory generated files are stored in
blockgaslimit = 100000000

# Check last block actually has the expected block gas limit
last_block_gaslimit = w3.eth.getBlock('latest').gasLimit
if blockgaslimit != last_block_gaslimit:
    print('Warning: Last block gas limit is != configured gas limit: ' + str(last_block_gaslimit) + ' != ' + str(blockgaslimit) )

### Generate ZoKrates Benchmark files:

In [4]:
# function for naming function parameters with chars. Converts number to char [a-z].
# 1 -> a, 26 -> z, 27 -> za
def name_inputs(num, acc=""):
    if num > 26:
        acc += chr(122)
        return name_inputs(num - 26, acc)
    else:
        return acc + chr(num + 96)

# generates valid ZoKrates functions with given number of inputs
def generate_function(input_num, directory):
    function_name = 'def main('
    function = "\n\t return "
    function_return = " -> ():"
    if input_num == 0:
        function_name = "def main()"
    elif input_num == 1:
        function_name = "def main(field a)"
    else:
        for i in range(1, input_num + 1):
            if i != input_num:
                function_name += ('field ' + name_inputs(i) + ', ')
            else:
                function_name += ('field ' + name_inputs(i) + ')')

    write_files(function_name + function_return + function, input_num, directory)
    
# writes .zok files
def write_files(content, input_num, directory):
    file = open(directory + '/' + str(input_num) + "-benchmark.zok", "w")
    file.write(content)
    file.close()

# writes .zok files
def clean_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)
    
# Cleans target directory
clean_dir(target_dir)

# copy special verifier for 0 inputs to target_dir
shutil.copyfile('./zero-input-verifier.sol', './target/zero-input-verifier.sol')

# Generates test .zok files based on max_input and step
inputlist = list(chain([0],range(step, max_inputs + 1, step)))

for i in inputlist:
    generate_function(i,target_dir)
print("Success! Files have been created")




Success! Files have been created


### Compile and deploy

In [ ]:
results_deploy = []
results_verify = []
    
def deploy_and_call(no_input):
    data = get_proof() 
#   Deploy Contract:
#   By default web3 will deploy the libraries needed in this contract, however we only measure deployment costs of the verifier contract.
    if no_input:
        bytecode = open('zero-input-verifier_sol_Verifier.bin', 'r').read()
        abi = open('zero-input-verifier_sol_Verifier.abi', 'r').read()
    else:
        bytecode = open('verifier_sol_Verifier.bin', 'r').read()
        abi = open('verifier_sol_Verifier.abi', 'r').read()
    contract = w3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = contract.constructor().transact({'gas': blockgaslimit})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    results_deploy.append(tx_receipt.gasUsed)
    contract = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
    
#   Call verify function
    if no_input:
        tx_hash = contract.functions.verifyTx(data[0][0], data[0][1], data[0][2]).transact({'gas': blockgaslimit})
    else:
        tx_hash = contract.functions.verifyTx(data[0][0], data[0][1], data[0][2], data[1]).transact({'gas': blockgaslimit})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    results_verify.append(tx_receipt.gasUsed)
    
def get_proof():
    with open('proof.json') as f:
        data = json.load(f)
        f.close()
        return (format_proof(data['proof']), format_inputs(data['inputs'])) 
    
# formats and returns proof data as decimal tuple
def format_proof(proof):
    converted = []
    for key in proof.keys():
        res = []
        converted.append(format_array(proof[key], res))       
    return tuple(converted)

#  convert inputs from hex to decimal
def format_inputs(inputs):
    res = []
    for i in inputs:
        res.append(int(i, 16))
    return res
    
# recursive function for converting the proof data from hex to decimal (needed because of web3.py) while keeping the nested nature of proof data
# only tested for g16 (because the only proof currently available with install script) but should work for any type of scheme
def format_array(arr, res = [], idx = None):
    for elem in arr:
        if type(elem) != list:
            if idx != None:
                res[idx].append(int(elem, 16))
            else:
                res.append(int(elem, 16))
        else:
            res.append([])
            format_array(elem, res, len(res)-1)
    return res
  
# set working directory to target_dir    
os.chdir(target_dir)

# copy special verifier for 0 inputs to target_dir
shutil.copyfile('../zero-input-verifier.sol', './zero-input-verifier.sol')
    
# compile zokrates programs, generate witness, deploy verifiaction contract and call    

for i in iter(inputlist):
    params = '3 '* i
    file = str(i) + '-benchmark.zok'
    !zokrates compile -i {i}-benchmark.zok --light
    !zokrates setup --light
    if i == 0 :
        !zokrates compute-witness --light
    else:
        !zokrates compute-witness -a {params} --light
    !zokrates generate-proof
    
    !zokrates export-verifier
#   Compiling contract via solc cli
    if i == 0:
        !solcjs --optimize --abi --bin zero-input-verifier.sol
        deploy_and_call(True)
    else:
        !solcjs --optimize --abi --bin verifier.sol
        deploy_and_call(False)

print("Results Deploy:")
print(results_deploy)
print("Difference Deploy per input:")
xd = np.array(results_deploy)
print(np.diff(xd))
print("Results Verify:")
print(results_verify)
print("Difference Verify per input:")
xd = np.array(results_verify)
print(np.diff(xd))

# reset working_dir
os.chdir('./..')


Compiling 0-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 1 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x01bef3e1adf6139084165015ac22931d7378d526ad4497e2220c83759440abf7", "0x263f2f93c40e985ef3c356149813deab9234d7646c44a80b5fc93c9d73bb6764"],
            "b": [["0x1b47cd4f7e22de2e86cdec29033260c77d48b886cbb5582460d2711f30f01ed1", "0x0d394304bac1e8ad9621da8b5d08b0b8536701f1c1e2e86bfd8a2ec61730e98b"], ["0x2b299e5cefb1a74ef64b824eea997b7f80b1fa99f668ec6461cff531f394d734", "0x268b541921df9b258d2cba24ec60eb0c65ad5440f96844a2745fcc66d09632a1"]],
            "c": ["0x13b62731fe5ca645878df3b5771ae26a214347b09f4e8358a9cfc9b660a5839f", "0x0cf55c28a5998a40a4aa20defbf84993cdbf8cb96e7d4553de28a8b2a9115650"]
        },
        "inputs": []
    }
Exporting verifier...
Finished exporting verifier.
zero-input-verifier.sol:450:5: Warning: Func

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

Compiling 60-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 61 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x200bb73fb55a96988c4404df2d2f75978683d2c28346775a2c8d31890c3e9af4", "0x12f5f4c319f5a06e2424e33679eda71cfac67476571780462670263800f69150"],
            "b": [["0x1b47cd4f7e22de2e86cdec29033260c77d48b886cbb5582460d2711f30f01ed1", "0x0d394304bac1e8ad9621da8b5d08b0b8536701f1c1e2e86bfd8a2ec61730e98b"], ["0x2b299e5cefb1a74ef64b824eea997b7f80b1fa99f668ec6461cff531f394d734", "0x268b541921df9b258d2cba24ec60eb0c65ad5440f96844a2745fc

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

Compiling 100-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 101 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x0d43554b2b77e28654525cf71a0a30a84e80406305aed6376095ec172bd78204", "0x0b3cfd4803a838b89f97fb851224c8399f59f697e5aea2b0d6f782db6bd3c5c5"],
            "b": [["0x1b47cd4f7e22de2e86cdec29033260c77d48b886cbb5582460d2711f30f01ed1", "0x0d394304bac1e8ad9621da8b5d08b0b8536701f1c1e2e86bfd8a2ec61730e98b"], ["0x2b299e5cefb1a74ef64b824eea997b7f80b1fa99f668ec6461cff531f394d734", "0x268b541921df9b258d2cba24ec60eb0c65ad5440f96844a2745

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

Compiling 120-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 121 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x1247d9468cbde56d61a605348d23bf342f242c6621ead7fe52234920ace02468", "0x2685293cb74706635ece4f9573e671084616a4e2f2fdccffe1fb6d1ccc9867cf"],
            "b": [["0x1b47cd4f7e22de2e86cdec29033260c77d48b886cbb5582460d2711f30f01ed1", "0x0d394304bac1e8ad9621da8b5d08b0b8536701f1c1e2e86bfd8a2ec61730e98b"], ["0x2b299e5cefb1a74ef64b824eea997b7f80b1fa99f668ec6461cff531f394d734", "0x268b541921df9b258d2cba24ec60eb0c65ad5440f96844a2745

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

Compiling 140-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 141 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x26c5be17f13c7462ede334408ccc6a58d625abedbb6869eaa1b9ecab04f36642", "0x1078f4c607dda190e31d8b98558fde22e1151c221ea6b62e2857afd55e25834a"],
            "b": [["0x1b47cd4f7e22de2e86cdec29033260c77d48b886cbb5582460d2711f30f01ed1", "0x0d394304bac1e8ad9621da8b5d08b0b8536701f1c1e2e86bfd8a2ec61730e98b"], ["0x2b299e5cefb1a74ef64b824eea997b7f80b1fa99f668ec6461cff531f394d734", "0x268b541921df9b258d2cba24ec60eb0c65ad5440f96844a2745

...

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

verifier.sol:543:1: Warning: Contract code size exceeds 24576 bytes (a limit introduced in Spurious Dragon). This contract may not be deployable on mainnet. Consider enabling the optimizer (with a low "runs" value!), turning off revert strings, or using libraries.
contract Verifier {
^ (Relevant source part starts here and spans across multiple lines).

Compiling 240-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 241 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x25a0becaa5c22c47b3d195655f76bda2b1fd28f5b7f9c765e5ba5600efd53f8f", 

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

verifier.sol:543:1: Warning: Contract code size exceeds 24576 bytes (a limit introduced in Spurious Dragon). This contract may not be deployable on mainnet. Consider enabling the optimizer (with a low "runs" value!), turning off revert strings, or using libraries.
contract Verifier {
^ (Relevant source part starts here and spans across multiple lines).

Compiling 260-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 261 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x2344518033a23c4a21b12e4937115ccb7b2059d053e3b33b8d4368319bf00859", 

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

verifier.sol:543:1: Warning: Contract code size exceeds 24576 bytes (a limit introduced in Spurious Dragon). This contract may not be deployable on mainnet. Consider enabling the optimizer (with a low "runs" value!), turning off revert strings, or using libraries.
contract Verifier {
^ (Relevant source part starts here and spans across multiple lines).

Compiling 280-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 281 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x2dfa0b5891219a795fd4a4259e94b8e4cddcd452281927ed3209c8730eb01a25", 

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

verifier.sol:543:1: Warning: Contract code size exceeds 24576 bytes (a limit introduced in Spurious Dragon). This contract may not be deployable on mainnet. Consider enabling the optimizer (with a low "runs" value!), turning off revert strings, or using libraries.
contract Verifier {
^ (Relevant source part starts here and spans across multiple lines).

Compiling 300-benchmark.zok

Compiled code written to 'out'
Number of constraints: 0
Performing setup...
Has generated 301 points
Setup completed.
Computing witness...

Witness: 

[]
Generating proof...
generate-proof successful: {
        "proof": {
            "a": ["0x0a18e0f43dadaa55140155ea69d336d0b6dbb251ee816679edb7c70c2ee7a018", 

Exporting verifier...
Finished exporting verifier.
verifier.sol:450:5: Warning: Function state mutability can be restricted to view
    function addition(G2Point memory p1, G2Point memory p2) internal returns (G2Point memory r) {
    ^ (Relevant source part starts here and spans across multiple lines).

verifier.sol:543:1: Warning: Contract code size exceeds 24576 bytes (a limit introduced in Spurious Dragon). This contract may not be deployable on mainnet. Consider enabling the optimizer (with a low "runs" value!), turning off revert strings, or using libraries.
contract Verifier {
^ (Relevant source part starts here and spans across multiple lines).



In [ ]:
def calc_trend_slope(x1, y1, x2, y2):
    print('Slope: ' + str((y2 - y1)/(x2 - x1)))
    return ((((y2 - y1)/(x2 - x1)) * (trend_factor * max_inputs - x1)) + y1) / 1000

    

### Deployment cost of contract

In [ ]:
import math
import matplotlib as mpl

# Use Latex for text rendering. Commented out to render default font without latex.
mpl.rc('text', usetex=True)
mpl.rc('font',**{'family':'serif','sans-serif':['Computer Modern']})

# calculate trendline slope
y_trend = calc_trend_slope(2*step, results_deploy[2], max_inputs, results_deploy[-1])

# blockgaslimit horizontal line
y_bgl = [10000, 10000]

# plot
fig, ax = plt.subplots()
ax.plot([max_inputs, max_inputs * trend_factor], [results_deploy[-1] / 1000, y_trend], '--', label="Trend", color="grey") # plot trend first, so it's behind main plot
ax.plot(inputlist, [x / 1000 for x in results_deploy], '-rs', label="Deployment", clip_on=False, zorder=100, markersize=5)
ax.plot([0, max_inputs * trend_factor], y_bgl, 'b:', linewidth=0.5) # show block gas limit
ax.axis([0, max_inputs*trend_factor, 0, results_deploy[-1]*2/1000])

# set x axis ticks
x_int = range(0, math.ceil(max_inputs*trend_factor),math.ceil(2*step)) # calculate x axis values
ax.set_xticks(x_int)

# format y axis numbers
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

# removing top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# adds a title and axes labels
# ax.set_title('Deployment Cost')
ax.set_xlabel('\# of Inputs')
ax.set_ylabel('k Gas')

# adds major gridlines
ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

# adds legend
ax.legend() 

# set size
fig.set_size_inches(11.69,8.27) # a4 landscape

# save as pdf
fig.savefig('exports/deployment.pdf', bbox_inches='tight')


### Transaction cost of Verification

In [ ]:
import math
import matplotlib as mpl

# Use Latex for text rendering. Commented out to render default font without latex.
mpl.rc('text', usetex=True)
mpl.rc('font',**{'family':'serif','sans-serif':['Computer Modern']})

# calculate trendline slope
y_trend = calc_trend_slope(0, results_verify[0], max_inputs, results_verify[-1])

# plot
fig, ax = plt.subplots()
ax.plot([max_inputs, max_inputs * trend_factor], [results_verify[-1] / 1000, y_trend], '--', label="Trend", color="grey") # plot trend first, so it's behind main plot
ax.plot(inputlist, [x / 1000 for x in results_verify], '-rs', label="Verification", clip_on=False, zorder=100, markersize=5)
ax.axis([0, max_inputs*trend_factor, 0, results_verify[-1]*2/1000])


# set x axis ticks
x_int = range(0, math.ceil(max_inputs*trend_factor),math.ceil(2*step)) # calculate x axis values
ax.set_xticks(x_int)

# removing top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# adds a title and axes labels
# ax.set_title('Deployment Cost')
ax.set_xlabel('\# of Inputs')
ax.set_ylabel('k Gas')

# adds major gridlines
ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

# adds legend
ax.legend() 

# format y axis numbers
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

# set size
fig.set_size_inches(10,6)

# save as pdf
fig.savefig('exports/verification.pdf', bbox_inches='tight')